In [1]:
import os
import csv
import numpy as np

In [12]:
action_features = np.load("../data/eastenders/action_features.npy")
with open("../data/eastenders/features.csv", "r") as fp:
    reader = csv.reader(fp, delimiter=' ')
    feature_idx_dict = {}
    action_label_dict = {}
    for r in reader:
        feature_idx = int(r[3])
        action_label = int(r[6])
        feature_idx_dict[feature_idx] = action_label
        if action_label != 99:
            if action_label in action_label_dict:
                action_label_dict[action_label].append(feature_idx)
            else:
                action_label_dict[action_label] = [feature_idx]
        else:
            continue

In [13]:
action_features.shape

(278958, 2304)

In [14]:
feature_idx_dict

{0: 99,
 1: 99,
 2: 99,
 3: 99,
 4: 99,
 5: 99,
 6: 99,
 7: 99,
 8: 99,
 9: 99,
 10: 99,
 11: 99,
 12: 99,
 13: 99,
 14: 99,
 15: 99,
 16: 99,
 17: 99,
 18: 99,
 19: 99,
 20: 99,
 21: 99,
 22: 99,
 23: 99,
 24: 99,
 25: 99,
 26: 99,
 27: 99,
 28: 99,
 29: 99,
 30: 99,
 31: 99,
 32: 99,
 33: 99,
 34: 99,
 35: 99,
 36: 99,
 37: 99,
 38: 99,
 39: 99,
 40: 99,
 41: 99,
 42: 99,
 43: 99,
 44: 99,
 45: 99,
 46: 99,
 47: 99,
 48: 99,
 49: 99,
 50: 99,
 51: 99,
 52: 99,
 53: 99,
 54: 99,
 55: 99,
 56: 99,
 57: 99,
 58: 99,
 59: 99,
 60: 99,
 61: 99,
 62: 99,
 63: 99,
 64: 99,
 65: 99,
 66: 99,
 67: 99,
 68: 99,
 69: 99,
 70: 99,
 71: 99,
 72: 99,
 73: 99,
 74: 99,
 75: 99,
 76: 99,
 77: 99,
 78: 99,
 79: 99,
 80: 99,
 81: 99,
 82: 99,
 83: 99,
 84: 99,
 85: 99,
 86: 99,
 87: 99,
 88: 99,
 89: 99,
 90: 99,
 91: 99,
 92: 99,
 93: 99,
 94: 99,
 95: 99,
 96: 99,
 97: 99,
 98: 11,
 99: 99,
 100: 9,
 101: 99,
 102: 5,
 103: 11,
 104: 99,
 105: 8,
 106: 4,
 107: 4,
 108: 99,
 109: 99,
 110: 11,
 111:

In [27]:
normalized = action_features / np.array([np.sqrt(np.sum(action_features ** 2, axis=1))]).T
np.sum(normalized[0] ** 2)

1.0

In [31]:
np.sum((normalized - normalized[0]) ** 2, axis=1)

array([0.        , 0.35988426, 0.41855234, ..., 0.30937943, 0.40062788,
       0.6127759 ], dtype=float32)

In [36]:
inner_product = np.array([np.dot(fv, action_features[0]) for fv in action_features])
norm = np.array([np.linalg.norm(fv) * np.linalg.norm(action_features[0]) for fv in action_features])
(- inner_product / norm).shape

(278958,)

In [41]:
def retrieval(query_idx, features, metric="l2+norm", limit=100):
    # 類似度の計算
    # 値が小さくなるほど類似度が高くなるように，cos類似度は値を-1倍している
    if metric == "l2+norm":
        normalized = action_features / np.array([np.sqrt(np.sum(action_features ** 2, axis=1))]).T
        similarity = np.sum((normalized - normalized[query_idx]) ** 2, axis=1)
    elif metric == "l2":
        similarity = np.sum((normalized - normalized[query_idx]) ** 2, axis=1)
    elif metric == "cosine":
        inner_product = np.array([np.dot(fv, features[query_idx]) for fv in features])
        norm = np.array([np.linalg.norm(fv) * np.linalg.norm(features[query_idx]) for fv in features])
        similarity = - inner_product / norm
    else:
        print("metric の値が不正です")
        return
    
    # 類似度スコア順にソートして limit で指定された上位のインデックスを返す
    ranking = np.argsort(similarity)
    return ranking[:limit]

In [44]:
def calc_ap(query_idx, rank, feature_idx_dict):
    gt = feature_idx_dict[query_idx]
    correct_num = 0
    total_num = 0
    precision_sum = 0.0
    for idx in rank:
        total_num += 1
        if gt == feature_idx_dict[idx]:
            # 正解ならば正解数と precision を加算していく
            correct_num += 1
            precision_sum += correct_num / total_num
        else:
            # 不正解なら何もしない
            continue
    # average precision を計算
    ap = precision_sum / correct_num
    return ap

In [45]:
rank = retrieval(98, action_features)
calc_ap(98, rank, feature_idx_dict)

0.3709591944886062

In [57]:
import sys
sys.path.append("/net/per610a/export/das18a/satoh-lab/iinuma/video_retrieval/diffusion")
import rank

ImportError: cannot import name 'Diffusion' from 'diffusion' (unknown location)

In [55]:
sys.path = sys.path[:12]

In [58]:
sys.path

['/net/per610a/export/das18a/satoh-lab/iinuma/video_retrieval/jupyter',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python37.zip',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python3.7',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python3.7/lib-dynload',
 '',
 '/home/iinuma/.local/lib/python3.7/site-packages',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages',
 '/net/per610a/export/das18a/satoh-lab/iinuma/video_retrieval/slowfast/detectron2_repo',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn-0.0-py3.7.egg',
 '/net/per610a/export/das18a/satoh-lab/iinuma/video_retrieval/slowfast',
 '/net/per610a/export/das18a/satoh-lab/iinuma/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/IPython/extensions',
 '/home/iinuma/.ipython',
 '/net/pe

In [63]:
os.getcwd()

'/net/per610a/export/das18a/satoh-lab/iinuma/video_retrieval/jupyter'

In [66]:
test_features = action_features[:5000]
test_query = action_features[:10]
np.save("../data/test_features", test_features)
np.save("../data/test_query", test_query)